In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import torch
from torchvision import datasets, transforms
from torch.utils.data import Subset, DataLoader

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [6]:
# Define transformation for MNIST (e.g., convert to tensor)
transform = transforms.Compose([transforms.ToTensor()
                               ,transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# Load the full MNIST training dataset
mnist_train = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
mnist_test = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)


# Create a DataLoader for the subset
test_loader = DataLoader(mnist_test, batch_size=64, shuffle=True)


# Select the first 600 images for training
subset_indices_1 = list(range(600))
mnist_subset_1 = Subset(mnist_train, subset_indices_1)

# Create a DataLoader for the subset
# train_loader_1 = DataLoader(mnist_subset_1, batch_size=64, shuffle=True)

# Select the first 600 images for training
subset_indices_2 = list(range(600,1200))
mnist_subset_2 = Subset(mnist_train, subset_indices_2)

# Create a DataLoader for the subset
# train_loader_2 = DataLoader(mnist_subset_2, batch_size=64, shuffle=True)

def get_data_loader(train_data, batch_size):
    return DataLoader(train_data, batch_size=batch_size, shuffle=True)


Files already downloaded and verified
Files already downloaded and verified


In [7]:
import torch.nn as nn
import torchvision.transforms as transforms
import torch.nn.functional as F

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 10, kernel_size=5)  # 3 input channels (for CIFAR-10), 10 output channels
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)  # 10 input channels, 20 output channels
        self.fc1 = nn.Linear(500, 50)  # We'll compute 500 dynamically based on input size
        self.fc2 = nn.Linear(50, 10)  # 10 output features (CIFAR-10 has 10 classes)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))  # RELU activation, max pooling
        x = F.relu(F.max_pool2d(self.conv2(x), 2))  # RELU activation, max pooling
        x = x.view(x.size(0), -1)  # Dynamically flatten the output
        x = F.relu(self.fc1(x))  # RELU activation
        x = self.fc2(x)
        return x


In [23]:
import torch.optim as optim

def evaluate(model,loader) -> float:
    correct, total = 0, 0
    model.to(device)
    model.eval()
    with torch.no_grad():
        for data in loader:
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total

def train(model,train_loader,batch_size=64,EPOCHS=5):
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
    model.train()
    for epoch in range(EPOCHS):
        running_loss = 0.0
        running_correct = 0.0
        model = model.to(device)
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            running_correct += (labels == predicted).sum().item()
        epoch_loss = running_loss / len(train_loader)
        epoch_accuracy = running_correct / (len(train_loader) * batch_size)
        if ((epoch + 1 ) % 5 == 0):
            test_accuracy = evaluate(model,test_loader)
            model.train()
            print(f"Epoch [{epoch+1}/{EPOCHS}]: Loss: {epoch_loss}, Train accuracy: {epoch_accuracy}, Test accuracy: {test_accuracy}")
        else:
            print(f"Epoch [{epoch+1}/{EPOCHS}]: Loss: {epoch_loss}, Train accuracy: {epoch_accuracy}")
    return  model

In [9]:
import torch
import copy

# Assume model_list is a list of models
def average_model_weights(model_list):
    avg_model_state_dict = copy.deepcopy(model_list[0])
    
    for key in avg_model_state_dict:
        avg_model_state_dict[key] = torch.zeros_like(avg_model_state_dict[key])

    num_models = len(model_list)
    for model_state_dict in model_list:
        for key in avg_model_state_dict:
            avg_model_state_dict[key] += model_state_dict[key]
    
    for key in avg_model_state_dict:
        avg_model_state_dict[key] /= num_models

    return avg_model_state_dict
    


In [24]:
def federated_learning(batch_size=50, init=True):
    train_loaders = [get_data_loader(mnist_subset_1, batch_size), get_data_loader(mnist_subset_2, batch_size)]

    def server_execute(t,E,K, init=True):
        if init:
            w0 = copy.deepcopy((SimpleCNN()).state_dict())
        else:
            w0 = None
        for i in range(t):
            weights = []
            for k in range(K):
                weights.append(ClientUpdate(E,k,w0))
            w0 = average_model_weights(weights)
            avg_model = SimpleCNN()
            avg_model.load_state_dict(w0)
            print(f"round {i} accuracy: {evaluate(avg_model,test_loader)}")
        return avg_model
        
    def ClientUpdate(E,k, w=None):
        model = SimpleCNN()
        if w is not None:
            model.load_state_dict(state_dict=w)
        model = train(model,train_loaders[k],batch_size=batch_size, EPOCHS=E)
        print(f"k={k}: {evaluate(model,test_loader)}")
        return model.state_dict()
    
    return server_execute(10,10,2, init=init)

Train model without Same init

In [25]:
modelInitF = federated_learning(init=False)

Epoch [1/10]: Loss: 2.3065064946810403, Train accuracy: 0.10666666666666667
Epoch [2/10]: Loss: 2.29205983877182, Train accuracy: 0.13833333333333334
Epoch [3/10]: Loss: 2.269938806692759, Train accuracy: 0.12833333333333333
Epoch [4/10]: Loss: 2.2447995146115622, Train accuracy: 0.13333333333333333
Epoch [5/10]: Loss: 2.189228594303131, Train accuracy: 0.18, Test accuracy: 0.2211
Epoch [6/10]: Loss: 2.074799875418345, Train accuracy: 0.2733333333333333
Epoch [7/10]: Loss: 1.9959490994612377, Train accuracy: 0.2866666666666667
Epoch [8/10]: Loss: 1.9037217100461323, Train accuracy: 0.305
Epoch [9/10]: Loss: 1.8959687451521556, Train accuracy: 0.31666666666666665
Epoch [10/10]: Loss: 1.8565955360730488, Train accuracy: 0.3383333333333333, Test accuracy: 0.2998
k=0: 0.2998
Epoch [1/10]: Loss: 2.302143315474192, Train accuracy: 0.12166666666666667
Epoch [2/10]: Loss: 2.2934609254201255, Train accuracy: 0.12833333333333333
Epoch [3/10]: Loss: 2.2791348695755005, Train accuracy: 0.135
Epoch

Train Models with same init

In [26]:
modelInitT = federated_learning(init=True)

Epoch [1/10]: Loss: 2.2984639604886374, Train accuracy: 0.125
Epoch [2/10]: Loss: 2.289472003777822, Train accuracy: 0.12833333333333333
Epoch [3/10]: Loss: 2.274119198322296, Train accuracy: 0.12833333333333333
Epoch [4/10]: Loss: 2.2441458900769553, Train accuracy: 0.12833333333333333
Epoch [5/10]: Loss: 2.1888291239738464, Train accuracy: 0.165, Test accuracy: 0.1874
Epoch [6/10]: Loss: 2.0760736962159476, Train accuracy: 0.23333333333333334
Epoch [7/10]: Loss: 1.95574352145195, Train accuracy: 0.28
Epoch [8/10]: Loss: 1.932986557483673, Train accuracy: 0.30666666666666664
Epoch [9/10]: Loss: 1.8736979762713115, Train accuracy: 0.295
Epoch [10/10]: Loss: 1.829034189383189, Train accuracy: 0.3333333333333333, Test accuracy: 0.2953
k=0: 0.2953
Epoch [1/10]: Loss: 2.305138885974884, Train accuracy: 0.08166666666666667
Epoch [2/10]: Loss: 2.2951716581980386, Train accuracy: 0.12333333333333334
Epoch [3/10]: Loss: 2.2741852601369223, Train accuracy: 0.15666666666666668
Epoch [4/10]: Loss

In [27]:
# compare the two models

print(f"Model with different initialisation: {evaluate(modelInitF,test_loader)}")

print(f"Model with same initialisation: {evaluate(modelInitT,test_loader)}")

Model with different initialisation: 0.3981
Model with same initialisation: 0.3826


Trying different batch_size

In [28]:
batch_sizes = [5, 10, 20, 30, 40]
models = []

for batch_size in batch_sizes:
    models.append(federated_learning(batch_size, init=True))

Epoch [1/10]: Loss: 2.2721957842508953, Train accuracy: 0.15166666666666667
Epoch [2/10]: Loss: 2.122735128800074, Train accuracy: 0.20666666666666667
Epoch [3/10]: Loss: 2.0596560527880987, Train accuracy: 0.21666666666666667
Epoch [4/10]: Loss: 2.0303317149480185, Train accuracy: 0.2633333333333333
Epoch [5/10]: Loss: 1.8732537165284158, Train accuracy: 0.32, Test accuracy: 0.2719
Epoch [6/10]: Loss: 1.801856929063797, Train accuracy: 0.3333333333333333
Epoch [7/10]: Loss: 1.7204065516591072, Train accuracy: 0.3433333333333333
Epoch [8/10]: Loss: 1.6466671307881673, Train accuracy: 0.3883333333333333
Epoch [9/10]: Loss: 1.6664632648229598, Train accuracy: 0.37833333333333335
Epoch [10/10]: Loss: 1.4435729826490085, Train accuracy: 0.465, Test accuracy: 0.2904
k=0: 0.2904
Epoch [1/10]: Loss: 2.265788061420123, Train accuracy: 0.14666666666666667
Epoch [2/10]: Loss: 2.1250873883565267, Train accuracy: 0.19833333333333333
Epoch [3/10]: Loss: 2.0461086044708887, Train accuracy: 0.2566666

In [29]:
for model, batch_size in zip(models, batch_sizes):
    print(f"Batch size: {batch_size}, Accuracy: {evaluate(model, test_loader)}")
print(f"Batch size: 50 , Accuracy: {evaluate(modelInitT, test_loader)}")

Batch size: 5, Accuracy: 0.1489
Batch size: 10, Accuracy: 0.2811
Batch size: 20, Accuracy: 0.3723
Batch size: 30, Accuracy: 0.3757
Batch size: 40, Accuracy: 0.3961
Batch size: 50 , Accuracy: 0.3826
